<a href="https://colab.research.google.com/github/AlexandreMinato/InvestimentoOpcoes/blob/main/AnaliseOpcoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.common.by import By
import selenium.webdriver.common.keys
from selenium.webdriver.common.keys import Keys
import random

In [ ]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
wd = webdriver.Chrome('chromedriver',options=options)


In [ ]:
def formatar_coluna(df_formatar, colunas):
  for coluna in colunas:
    df_formatar[coluna] = df_formatar[coluna]/100 
    

In [ ]:
def capturar_dados_opc(ticker, colunas_df):
    url = "https://opcoes.net.br/opcoes/bovespa/" + ticker
    wait = WebDriverWait(wd, 5000)
    time.sleep(2)
    wd.get(url)
    time.sleep(2) #AGUARDA o script do Datatable rodar
    html = wd.page_source
    soup = BeautifulSoup(html, 'lxml')
    table = soup.find_all('table', { "id" : "tblListaOpc" }) #tblListaOpc
    df_result = pd.read_html(str(table))
    df_result[0].columns = colunas_df
    df_result = df_result[0]
    df_result.set_index('ticker')
    colunas_formatar = ['strike',  'vol_implicita', 'delta', 'gamma', 'theta_R$', 'theta', 'vega']
    formatar_coluna(df_formatar=df_result, colunas=colunas_formatar)
    df_result.insert(1,'ativo', ticker) 
    return df_result  

In [ ]:
posicoes = ['BBDC4','EGIE3','ITSA4','BEEF3','COGN3','BBAS3','GOAU4','CVCB3','SBSP3','EVEN3','ARZZ3','PETR4','SULA11','B3SA3','CIEL3','TRPL4','IRBR3']
#posicoes = ['BBDC4','EGIE3','ITSA4']
colunas = ['ticker', 'tipo','FM','mod','strike','a_i_OTM','ultimo','variacao','data','num_negocios','volume_financeiro','vol_implicita','delta','gamma','theta_R$', 'theta', 'vega']
df_consolidado = pd.DataFrame(columns=colunas)

In [ ]:
for posicao in posicoes:
  df_temp = capturar_dados_opc(posicao, colunas)
  time.sleep(2)
  df_consolidado = df_consolidado.append(df_temp, ignore_index=True)

In [ ]:
def capturar_valor_black_scholes(tiker_name, ticker_ativo):
  dfs = []
  urlopcoes = 'https://opcoes.net.br/calculadora/dadosopcao?ticker=' + tiker_name
  print(urlopcoes)
  
  
  retries = 1
  success = False
  while not success:
    try:
        dados = requests.get(urlopcoes,  headers={'User-Agent': 'Mozilla/5.0'})
        print(dados)
        content = json.loads(dados.text)
        print(content)
        content.update({"ticker": tiker_name, "ativo": ticker_ativo})
        # dfs.append(pd.DataFrame([content]))
        # dfjson = pd.concat(dfs, ignore_index=True, sort=False)
        dfs = pd.json_normalize([content])
        success = True
    except Exception as e:
        print('tentando novamente por erro 429 (muitas requisições).' )
        time.sleep(random.randint(10, 20))
        retries += 1
  
  return dfs

In [ ]:
result = pd.DataFrame()
for index, row in df_consolidado.iterrows():
  # df_bs = df_bs.append(capturar_valor_black_scholes(row['ticker']), ignore_index=True)
  time.sleep(random.randint(3, 15))
  _result = capturar_valor_black_scholes(row['ticker'], row['ativo'])
  result = pd.concat([result, _result])


In [ ]:
result.set_index('ticker')

In [ ]:
df_consolidado.set_index('ticker')

,tipo,FM,mod,strike,a_i_OTM,ultimo,variacao,data,num_negocios,volume_financeiro,vol_implicita,delta,gamma,theta_R$,theta,vega,ativo
ticker,,,,,,,,,,,,,,,,,
BBDCJ119,CALL,NaN,E,18.99,ITM,177,-585,24/09/2021,1,9.912,3.34,86.53,13.38,-1.65,-0.93,105.13,BBDC4
BBDCV119,PUT,NaN,E,18.99,OTM,19,"+26,67",24/09/2021,56,28.966,3.51,-19.65,16.55,-1.55,-8.16,132.12,BBDC4
BBDCJ141,CALL,NaN,A,19.24,ITM,143,-1118,24/09/2021,4,10.107,3.49,78.53,17.43,-2.11,-1.48,140.33,BBDC4
BBDCV141,PUT,NaN,E,19.24,OTM,24,"+33,33",24/09/2021,42,28.74,3.40,-24.10,19.22,-1.68,-7.00,148.50,BBDC4
BBDCJ146,CALL,NaN,E,19.49,ITM,137,"+39,80",23/09/2021,30,54.768,3.31,77.27,18.16,-2.03,-1.48,151.29,BBDC4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IRBRJ540,CALL,✔,A,5.40,OTM,6,000,24/09/2021,38,11.052,4.04,22.79,63.62,-0.54,-9.00,35.53,IRBR3
IRBRV540,PUT,NaN,E,5.40,ITM,45,-1176,24/09/2021,4,8.183,4.41,-73.74,62.57,-0.50,-1.11,38.50,IRBR3
IRBRJ550,CALL,NaN,E,5.50,OTM,6,"+20,00",24/09/2021,26,2.065,4.52,20.93,54.01,-0.57,-9.50,33.89,IRBR3


In [ ]:
frames = [result, df_consolidado]
result_union = pd.concat(frames)